In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=cdad65a9f8db5dc08f40dba8fb72c30deddaadbdcea1cfdad9f60e20658a07fd
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [3]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

In [4]:
os.chdir("/content/drive/My Drive/IRProject")

In [5]:
# Read the csv file
df = pd.read_csv('covid_df_20000.csv')
df

,Unnamed: 0,doc_id,title,abstract,text_body
0,1,5ac635d21fdb1726f21d7090a233dd2b5ffdc1b5,"Long Term Predictors of Breathlessness, Exerci...",J o u r n a l P r e -p r o o f 2,With over 246 million Coronavirus disease 2019...
1,2,71d93a6452061c57ae1532991d4a20cd6fc6fcb3,Emergence of universality in the transmission ...,The complexities involved in modelling the tra...,• Even though the pattern of disease spread is...
2,3,635ba7eca764f7caaa82904197c40a20111ec940,Supporting families to protect child health: P...,Supportive parenting is critical for promoting...,a1111111111 a1111111111 a1111111111 a111111111...
3,4,0282d2f9eb65318e40db21efbe6172ce16c8acf5,Perspectives on the Early Quality of Evidence ...,Background: The severe acute respiratory syndr...,The severe acute respiratory syndrome coronavi...
4,5,d96113a2d8691d3b1aee5fd1b5d30241f2b2a633,Quantify the role of superspreaders -opinion l...,Effective communication of accurate informatio...,We design a mathematical model to quantify the...
...,...,...,...,...,...
19995,19996,ffdc50d239f8e531dc159993723c0cbc176a632e,Comparative analysis of antibody-and lipid-bas...,Multiplexing of samples in single-cell RNA-seq...,Recent advances in single-cell and single-nucl...
19996,19997,b37bf3ab1d814f853a42cd8958d7390b42343bd2,NaN,NaN,To the Editor:\n We read with interest the art...
19997,19998,2c70b8caf708c5c1b0c9b811a80179061274cff7,Nervensystem,NaN,zeichen diagnostisch eine Rolle spielen. Am Ko...
19998,19999,1831d9c801bcbc3388eb2aeb7d48aed3c2c690e9,Journal Pre-proof The sub-specialty of Foot an...,NaN,The sub-specialty of Foot and Ankle is evolvin...


In [6]:
# Concatenate the title and abstract columns to form a new column called 'full_text'
df['title_body'] = df['title'].fillna('') + ' ' + df['text_body'].fillna('')
df

,Unnamed: 0,doc_id,title,abstract,text_body,title_body
0,1,5ac635d21fdb1726f21d7090a233dd2b5ffdc1b5,"Long Term Predictors of Breathlessness, Exerci...",J o u r n a l P r e -p r o o f 2,With over 246 million Coronavirus disease 2019...,"Long Term Predictors of Breathlessness, Exerci..."
1,2,71d93a6452061c57ae1532991d4a20cd6fc6fcb3,Emergence of universality in the transmission ...,The complexities involved in modelling the tra...,• Even though the pattern of disease spread is...,Emergence of universality in the transmission ...
2,3,635ba7eca764f7caaa82904197c40a20111ec940,Supporting families to protect child health: P...,Supportive parenting is critical for promoting...,a1111111111 a1111111111 a1111111111 a111111111...,Supporting families to protect child health: P...
3,4,0282d2f9eb65318e40db21efbe6172ce16c8acf5,Perspectives on the Early Quality of Evidence ...,Background: The severe acute respiratory syndr...,The severe acute respiratory syndrome coronavi...,Perspectives on the Early Quality of Evidence ...
4,5,d96113a2d8691d3b1aee5fd1b5d30241f2b2a633,Quantify the role of superspreaders -opinion l...,Effective communication of accurate informatio...,We design a mathematical model to quantify the...,Quantify the role of superspreaders -opinion l...
...,...,...,...,...,...,...
19995,19996,ffdc50d239f8e531dc159993723c0cbc176a632e,Comparative analysis of antibody-and lipid-bas...,Multiplexing of samples in single-cell RNA-seq...,Recent advances in single-cell and single-nucl...,Comparative analysis of antibody-and lipid-bas...
19996,19997,b37bf3ab1d814f853a42cd8958d7390b42343bd2,NaN,NaN,To the Editor:\n We read with interest the art...,To the Editor:\n We read with interest the ar...
19997,19998,2c70b8caf708c5c1b0c9b811a80179061274cff7,Nervensystem,NaN,zeichen diagnostisch eine Rolle spielen. Am Ko...,Nervensystem zeichen diagnostisch eine Rolle s...
19998,19999,1831d9c801bcbc3388eb2aeb7d48aed3c2c690e9,Journal Pre-proof The sub-specialty of Foot an...,NaN,The sub-specialty of Foot and Ankle is evolvin...,Journal Pre-proof The sub-specialty of Foot an...


In [7]:
# Load a pre-trained BERT model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Generate sentence embeddings for each full text in the DataFrame
embeddings = model.encode(df['title_body'].tolist(), convert_to_tensor=True)

In [8]:
# Define a function for searching the DataFrame using a query string
def search(query, embeddings, top_n=10):
    # Encode the query string to generate a query embedding
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the cosine similarity between the query embedding and the embeddings of each full text
    cos_scores = torch.nn.functional.cosine_similarity(query_embedding, embeddings).cpu().numpy()

    # Get the indices of the top n scores in descending order
    top_indices = cos_scores.argsort()[-top_n:][::-1]

    # Sort the cosine similarity scores array in a descending order
    cos_scores_sorted = np.sort(cos_scores)
    cos_scores_sorted_descending = cos_scores_sorted[::-1]

    # Return the DataFrame rows corresponding to the top n scores and the numpy array of the top n scores
    return df.iloc[top_indices], cos_scores_sorted_descending


In [9]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: vaccine effectiveness
Top 10 results for the query 'vaccine effectiveness':

Score: 0.6528316736221313
Title: pharmaceutics Nanoparticles to Improve the Efficacy of Vaccines
Body Text:
 A vaccine, providing active and protective immunity against a target disease, contains an agent that originated from and/or resembles a disease-causing microorganism. It is often made from a weakened or inactivated microbe, its toxins, or one of its nucleotides, peptides or proteins. Vaccines can be prophylactic to ameliorate or better prevent the effects of a wild-type pathogen, or therapeutic against likely cancers. To date, the World Health Organization lists twenty-seven preventable infections for which vaccines are available [1] -far less than what our society needs.
 Vaccination, a process of introducing foreign antigenic material(s) in order to activate a host immune system, has been a key strategy to control diseases and improve quality of life in humans and animals. Des

In [10]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: covid-19 transmission
Top 10 results for the query 'covid-19 transmission':

Score: 0.603185772895813
Title: Shared genomic architecture between COVID-19 severity and numerous clinical and physiologic parameters revealed by LD score regression analysis
Body Text:
 We downloaded the GWAS summary statistics (COVID19-hg GWAS meta-analyses round 5, released on January 18, 2021; https:// www. covid 19hg. org/ resul ts/ r5/) from the COVID-19 Host Genetics Initiative (COVID-19 HGI) [11] [12] [13] , comprising (1) A2 (critical illness) 13 : 4,606 very severe, respiratory-confirmed COVID-19 patients versus 702,801 population-based controls (A2_ALL_eur_leave_23andme) and (2) B2 (hospitalization) 13 : 9,373 hospitalized COVID-19 patients versus 1,197,256 population-based controls (B2_ALL_eur_leave_23andme) ( Table 1 and  Supplementary Tables 1 and 2 ). While the summary statistics of COVID-19-hg GWAS meta-analyses across multiple populations have been deposited at the CO

# Let us repeat the same steps for the 'abstract' column instead of the 'text_body' and see which method yields higher similarity scores.

In [11]:
# Concatenate the title and abstract columns to form a new column called 'full_text'
df['title_abstract'] = df['title'].fillna('') + ' ' + df['abstract'].fillna('')
df

,Unnamed: 0,doc_id,title,abstract,text_body,title_body,title_abstract
0,1,5ac635d21fdb1726f21d7090a233dd2b5ffdc1b5,"Long Term Predictors of Breathlessness, Exerci...",J o u r n a l P r e -p r o o f 2,With over 246 million Coronavirus disease 2019...,"Long Term Predictors of Breathlessness, Exerci...","Long Term Predictors of Breathlessness, Exerci..."
1,2,71d93a6452061c57ae1532991d4a20cd6fc6fcb3,Emergence of universality in the transmission ...,The complexities involved in modelling the tra...,• Even though the pattern of disease spread is...,Emergence of universality in the transmission ...,Emergence of universality in the transmission ...
2,3,635ba7eca764f7caaa82904197c40a20111ec940,Supporting families to protect child health: P...,Supportive parenting is critical for promoting...,a1111111111 a1111111111 a1111111111 a111111111...,Supporting families to protect child health: P...,Supporting families to protect child health: P...
3,4,0282d2f9eb65318e40db21efbe6172ce16c8acf5,Perspectives on the Early Quality of Evidence ...,Background: The severe acute respiratory syndr...,The severe acute respiratory syndrome coronavi...,Perspectives on the Early Quality of Evidence ...,Perspectives on the Early Quality of Evidence ...
4,5,d96113a2d8691d3b1aee5fd1b5d30241f2b2a633,Quantify the role of superspreaders -opinion l...,Effective communication of accurate informatio...,We design a mathematical model to quantify the...,Quantify the role of superspreaders -opinion l...,Quantify the role of superspreaders -opinion l...
...,...,...,...,...,...,...,...
19995,19996,ffdc50d239f8e531dc159993723c0cbc176a632e,Comparative analysis of antibody-and lipid-bas...,Multiplexing of samples in single-cell RNA-seq...,Recent advances in single-cell and single-nucl...,Comparative analysis of antibody-and lipid-bas...,Comparative analysis of antibody-and lipid-bas...
19996,19997,b37bf3ab1d814f853a42cd8958d7390b42343bd2,NaN,NaN,To the Editor:\n We read with interest the art...,To the Editor:\n We read with interest the ar...,
19997,19998,2c70b8caf708c5c1b0c9b811a80179061274cff7,Nervensystem,NaN,zeichen diagnostisch eine Rolle spielen. Am Ko...,Nervensystem zeichen diagnostisch eine Rolle s...,Nervensystem
19998,19999,1831d9c801bcbc3388eb2aeb7d48aed3c2c690e9,Journal Pre-proof The sub-specialty of Foot an...,NaN,The sub-specialty of Foot and Ankle is evolvin...,Journal Pre-proof The sub-specialty of Foot an...,Journal Pre-proof The sub-specialty of Foot an...


In [12]:
# Generate sentence embeddings for each full text in the DataFrame
embeddings = model.encode(df['title_abstract'].tolist(), convert_to_tensor=True)

In [13]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: vaccine effectiveness
Top 10 results for the query 'vaccine effectiveness':

Score: 0.8376752734184265
Title: Vaccines and Prevention
Body Text:
 I was in my first week as a newly graduated doctor when I encountered a vaccine-skeptic -a patient who was convinced vaccines were dangerous and could weaken the human race. I remember thinking -as a doctor, I am supposed to take a detailed history, but I hope you forgive me if I switch off just for a little bit, because right now isn't the time for this debate. You learn something at medical school and immediately try to tell people about it, but you can't convince them -not easily anyway. Especially if you have been indoctrinated with factual information along the lines of this 2 : an estimated 1.5 million children die each year-one every 20 seconds-from vaccine-preventable diseases such as diarrhea and pneumonia.
 When I was thirteen, I had my first experience of outcomes that such beliefs can lead to -one of my be

In [14]:
if __name__ == "__main__":
    # Prompt the user to enter a query string
    query = input("Please enter your query: ")
    
    # Set the number of top results to display
    top_n = 10
    
    # Call the search function to find the top results
    results = search(query, embeddings, top_n)

    # Print the top results to the console
    print(f"Top {top_n} results for the query '{query}':")

    for (index, row), score in zip(results[0].iterrows(), results[1]):
        print(f'\nScore: {score}')
        print(f"Title: {row['title']}\nBody Text:\n {row['text_body']}")

Please enter your query: covid-19 transmission
Top 10 results for the query 'covid-19 transmission':

Score: 0.8505504131317139
Title: Updates of COVID-19
Body Text:
 Coronaviruses (CoVs) have been linked with major disease outbreaks in East Asia and the Middle East. They cause severe acute respiratory syndrome (SARS) and the Middle East respiratory syndromes (MERS). The disease has been found in 2002 and 2012. At present, a novel CoV has emerged in 2019-2020 which may cause a health hazard that specified the ongoing pandemic all over the world (Rodriguez-Morales et al. 2020 ). On 11 February 2020, the World Health Organization (WHO) announced the official designation for this current CoV-associated disease to be "COVID-19." These newly evolved CoVs have been posing a significant threat to global public health (Wellcome 2020) .
 COVID-19 is caused by a novel SARS-CoV-2 (severe acute respiratory syndrome coronavirus-2) reported in 2019 (Wang et al. 2020a, b, c) . This virus has been ori